# Lockdown Timing Map

## Code book

### For Lockdown data

#### Variables and their units

- Our dataframe (called data) contains following columns:
    - Country/Region -> Names
    - Province -> Names
    - Date -> Datetime
    - Type -> categorical Full/Partial/None
    - Reference -> link to governmental homepage or newspaper


#### Summary choices made

- We discarded:
    - Reference
    - Province (for most countries)
    - Type (in other script processed)
    
- Since we have in the json file higher geographical resolution for China, USA, Australia and higher geographical resolution in our lockdown data we decided to show higher geographical resolution for these three countries
    - Therefore we had to create one seperate additional dataframe for each Country 
        

### For geographical data (json)

#### Variables and their units

- Each country (or for CN, US, AU each Province) name is associated with polygons 

#### Summary choices made

- functions were written that use the name of the geographical entity (from json file) and this name is the index of the dataframe (from Lockdown data) with the information about when a lockdown happened and a colormap assigns a color to this specific datetime data.

## Imports

In [56]:
import pandas as pd
import numpy as np
import folium
import json
import requests

## Read in data

Data from: https://www.kaggle.com/jcyzag/covid19-lockdown-dates-by-country

In [57]:
data = pd.read_csv('./Lockdown_dates/countryLockdowndates.csv')

In [58]:
# Opening JSON file 
world = open('countries.geo.json',)
china = open('china.geojson',)
australia = open('australia.geojson',)
US_url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'

# returns JSON object as  
# a dictionary 
geodata = json.load(world)
geodata_china = json.load(china)
geodata_australia = json.load(australia)
us_states = f'{US_url}/us-states.json'

# Closing file 
world.close() 
china.close()
australia.close()

## Data wrangling

In [59]:
#change names so that the names align with the json files
data.replace('Congo (Kinshasa)', 'Democratic Republic of the Congo', inplace=True)
data.replace('Cote dIvoire', 'Ivory Coast', inplace=True)
data.replace('Bahamas','The Bahamas', inplace=True)
data.replace('Congo (Brazzaville)','Republic of the Congo', inplace=True)
data.replace('Czechia','Czech Republic', inplace=True)
data.replace('Guinea-Bissau','Guinea Bissau', inplace=True)
data.replace('Korea, South','South Korea', inplace=True)
data.replace('North Macedonia', 'Macedonia', inplace=True)
data.replace('Burma', 'Myanmar', inplace=True)
data.replace('Serbia','Republic of Serbia', inplace=True)
data.replace('Timor-Leste','East Timor', inplace=True)
data.replace('Tanzania','United Republic of Tanzania', inplace=True)
data.replace('West Bank and Gaza', 'West Bank', inplace=True)

In [60]:
#two entries are not possible for folium remove both, add one
#Denmark    2020-03-11 and Denmark    2020-03-12
data.drop(data[(data['Country/Region']=='Denmark') & (data['Date']=='12/03/2020')].index, inplace=True)

In [61]:
data.drop(data[(data['Country/Region']=='Canada') & (data['Date']!='19/03/2020')].index, inplace=True)

In [62]:
data['Date'] = pd.to_datetime(data['Date'], format='%d/%m/%Y')
data['Date'] = data['Date'].dt.date
data = data.sort_values(by="Date", ascending = True)

In [63]:
dates = data['Date']
dates= list(dates)

In [64]:
#create additional dataframes for countries that have multiple entries

data_AU = data.loc[data['Country/Region']== 'Australia']
data_AU = data_AU.set_index('Province')

data_CN = data.loc[(data['Country/Region']== 'China')]
data_CN = data_CN.set_index('Province')

data_FR = data.loc[data['Country/Region']== 'France']

data_NL = data.loc[data['Country/Region']== 'Netherlands']

data_UK = data.loc[data['Country/Region']== 'United Kingdom']

data_US = data.loc[data['Country/Region']== 'US']
data_US = data_US.set_index('Province')

In [65]:
#append Taiwan to China since Taiwan is part of China in the json file
temp = data.loc[data['Country/Region']== 'Taiwan*']
temp['Province'] = 'Taiwan'
temp['Country/Region'] = 'China'
temp = temp.set_index('Province')
data_CN = data_CN.append(temp)
data_map = data

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [66]:
#making a duplicate of data without AU,CN,US since the are treated seperately for the Map
#delete FR, NL, UK as well and add only the mainland in a second step so that the series is unambiguous for the map
data_map = data_map[~data_map['Country/Region'].isin(['Australia', 'China', 'US', 'France', 'Netherlands', 'United Kingdom', 'Taiwan*'])]

data_map = data_map.append(data_NL.iloc[1])
data_map = data_map.append(data_FR.iloc[0])
data_map = data_map.append(data_UK.iloc[2])

In [67]:
#only select Time of Lockdown for the individual dataframes (world, AU, CN, US)
data_map_series = data_map.set_index('Country/Region')['Date']
data_AU_series = data_AU['Date']
data_CN_series = data_CN['Date']
idx = data_CN_series.index.values
#change name so that it aligns with the name in the json file
idx[13] = 'Heilongjian'
data_CN_series.index = idx
data_US_series = data_US['Date']

## Plotting with Folium

In [68]:
import branca.colormap as cm

#red january
#orange from start february to start march
#yellow from start march to 11th march
#green from 11th march to 23th march
#blue from 23th march to 6th april

step = cm.StepColormap(
    ['red', 'orange', 'yellow', 'green', 'blue'],
    vmin=min(dates), vmax=max(dates),
    index=[min(dates), dates[32] , dates[36], dates[50], dates[150], max(dates)],
    caption='step')

In [69]:
#writing functions that extract time of lockdown based on read country names from the json file


def get_color_AU_step(feature):
    value = data_AU_series[feature['properties']['STATE_NAME']]
    if pd.isna(value):
        return '#f81894' 
    else:
        return step(value)   

def get_color_CN_step(feature):
    if feature['properties']['name'] == 'Inner Mongolia':
        return '#8c8c8c'
    if feature['properties']['name'] == 'Jiangsu':
        return step(min(dates))
    value = data_CN_series[feature['properties']['name']]
    if pd.isna(value):
        return '#f81894'
    else:
        return step(value)

def get_color_US_step(feature):
    value = data_US_series[feature['properties']['name']]
    if pd.isna(value):
        return '#f81894'
    else:
        return step(value)    

    
def get_color_step(feature):
    if feature['properties']['name'] in ['Antarctica', 'French Southern and Antarctic Lands', 'Australia',
                                        'Burundi', 'Bermuda', 'China', 'Northern Cyprus',
                                        'Falkland Islands', 'French Guiana', 'Lesotho', 'Malawi', 'New Caledonia',
                                        'Puerto Rico', 'North Korea', 'Western Sahara', 'South Sudan',
                                        'Solomon Islands', 'Somaliland', 'Swaziland', 'Tajikistan','Turkmenistan',
                                        'Taiwan', 'United States of America', 'Vanuatu', 'Yemen']:
        return '#8c8c8c'
    value = data_map_series[feature['properties']['name']]
    if pd.isna(value):
        return '#f81894'
    else:
        return step(value)

In [70]:
#loads worldmap, first number being Latitude (north/south) , second Longitude (east/west), zoom_start = 1 far, 5 close
m = folium.Map([43, 140], zoom_start=2)

#highlightes the geographic entities form the geo_json data
folium.GeoJson(
    geodata,
    name='data_map',
    style_function=lambda feature: {
        'fillColor': get_color_step(feature),
        'color': 'black',
        'weight': 1,
        'dashArray': '5, 5',
        'fillOpacity': 0.9,
    }).add_to(m)



folium.GeoJson(
    us_states,
    name='data_US_series',
    style_function=lambda feature: {
        'fillColor': get_color_US_step(feature),
        'color': 'black',
        'weight': 1,
        'dashArray': '5, 5',
        'fillOpacity': 0.9,
    }).add_to(m)

folium.GeoJson(
    geodata_china,
    name='data_CN',
    style_function=lambda feature: {
        'fillColor': get_color_CN_step(feature),
        'color': 'black',
        'weight': 1,
        'dashArray': '5, 5',
        'fillOpacity': 0.9,
    }).add_to(m)


folium.GeoJson(
    geodata_australia,
    name='data_AU',
    style_function=lambda feature: {
        'fillColor': get_color_AU_step(feature),
        'color': 'black',
        'weight': 1,
        'dashArray': '5, 5',
        'fillOpacity': 0.9,
    }
).add_to(m)

step.add_to(m)


    
    
from branca.element import Template, MacroElement

template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>jQuery UI Draggable - Default functionality</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>Start Lockdown</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:red;opacity:1;'></span>January</li>
    <li><span style='background:orange;opacity:1;'></span>February</li>
    <li><span style='background:yellow;opacity:1;'></span>1st March - 11th March</li>
    <li><span style='background:green;opacity:1;'></span>11th March - 23th March</li>
    <li><span style='background:blue;opacity:1;'></span>23th March - 6th April</li>
    <li><span style='background:Magenta;opacity:1;'></span>no Lockdown</li>
    <li><span style='background:grey;opacity:1;'></span>no information</li>

  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)

m.get_root().add_child(macro)


m